# Queries within queries

## Overview

In this activity, I explore SQL subqueries and learn how to use them to build more complex queries. Subqueries are often used in FROM and WHERE clauses. The innermost query runs first, and its results are passed to the outer query.

## Dataset

I will use the BigQuery public dataset called `new_york` with the full path `bigquery-public-data.new_york`. This dataset contains many tables and for this activity, I will only be using the following tables:

- citibike_stations
- citibike_trips

## Exploring: Use a subquery in a SELECT statement

I will compare the number of bikes available at a particular station to the overall average number of bikes available at all stations by executing the following query:

In [ ]:
/* Outer query to obtain number of bikes
   available at a particular station */

SELECT
	station_id,
	num_bikes_available,
	
	/* Subquery (inner query) to obtain
	   average number of bikes available */
	(SELECT
		AVG(num_bikes_available)
	FROM bigquery-public-data.new_york.citibike_stations)
    AS avg_num_bikes_available -- Subquery alias

FROM bigquery-public-data.new_york.citibike_stations;

The query returns a table containing the station id, the number of bikes available at each station and the overall average number of bikes available for all stations, as shown below:

![Number of bikes at each station](c05m03-query-select.png 'Number of bikes at each station')

## Exploring: Use a subquery in a FROM statement

